# Proyecto: Análisis de Mamografías y Predicción de Cáncer de Mama


## 1. Introducción
El cáncer de mama es una de las principales causas de muerte entre mujeres en todo el mundo. La detección temprana y el diagnóstico preciso son cruciales para mejorar las tasas de supervivencia.
Este proyecto se centra en el análisis de mamografías para identificar patrones que puedan ayudar a distinguir entre tumores benignos y malignos. Utilizaremos técnicas de análisis estadístico, modelos predictivos y reducción de dimensionalidad para lograr este objetivo.


## 2. Objetivo
Analizar parámetros derivados de mamografías para distinguir entre tumores benignos y malignos.

### Metodología

1. **Análisis Exploratorio de Datos (EDA)**: Visualización y análisis de la distribución de características, correlaciones y patrones en los datos.
2. **Preprocesamiento de Datos**: Limpieza y transformación de los datos para prepararlos para el modelado, incluyendo la normalización y la reducción de dimensionalidad.
3. **Modelado Predictivo**: Implementación de modelos de machine learning para clasificar tumores como benignos o malignos.
4. **Evaluación de Modelos**: Uso de métricas de rendimiento para evaluar la efectividad de los modelos y realizar ajustes según sea necesario.


## 3. Dataset y Carga

El dataset puede ser descargado mediante la siguiente celda de código:
